In [1]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import transformers
from transformers import AutoModel,BertTokenizerFast



In [20]:
device = torch.device('cpu')
device

device(type='cpu')

In [3]:
df = pd.read_csv('emails.csv')
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [4]:
df.spam.value_counts(normalize=True)

0    0.761173
1    0.238827
Name: spam, dtype: float64

In [5]:
train_text,temp_text,train_label,temp_label = train_test_split(df.text,
                                                               df.spam,
                                                               test_size=0.3,
                                                               stratify=df.spam)

In [6]:
val_text,test_text,val_label,test_label = train_test_split(temp_text,
                                                           temp_label,
                                                           test_size=0.5,
                                                           stratify=temp_label)

In [10]:
# bert = AutoModel.from_pretrained('bert-base-uncased')

tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
# text = ['This is a sample test.','This is to check bert tokenizing']
# sent_id = tokenizer.batch_encode_plus(text,padding=True,return_token_type_ids=False)
# sent_id

{'input_ids': [[101, 2023, 2003, 1037, 7099, 3231, 1012, 102, 0], [101, 2023, 2003, 2000, 4638, 14324, 19204, 6026, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [ ]:
# seq_len = [len(i.split()) for i in train_text]

In [ ]:
# pd.Series(seq_len).value_counts()

101     28
75      28
95      26
203     26
81      25
        ..
1424     1
1472     1
1482     1
1484     1
2        1
Length: 937, dtype: int64

In [8]:
max_seq_len = 256

In [ ]:
tokens_train = tokenizer.batch_encode_plus(
      train_text.to_list(),
      max_length = max_seq_len,
      pad_to_max_length = True,
      truncation=True,
      return_token_type_ids=False

)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
tokens_val = tokenizer.batch_encode_plus(
      val_text.to_list(),
      max_length = max_seq_len,
      pad_to_max_length = True,
      truncation=True,
      return_token_type_ids=False

)

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1770: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [11]:
tokens_test = tokenizer.batch_encode_plus(
      test_text.to_list(),
      max_length = max_seq_len,
      pad_to_max_length = True,
      truncation=True,
      return_token_type_ids=False

)

/home/prateek/vpy/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1764: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_label.to_list())

In [ ]:
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_label.to_list())

In [12]:
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])
test_y = torch.tensor(test_label.to_list())

In [ ]:
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler

In [ ]:
batch_size = 32

train_data = TensorDataset(train_seq,train_mask,train_y)

train_sampler = RandomSampler(train_data)

train_dataloader = DataLoader(train_data,sampler=train_sampler,batch_size=batch_size)

In [ ]:
val_data = TensorDataset(val_seq,val_mask,val_y)

val_sampler = SequentialSampler(val_data)

val_dataloader = DataLoader(val_data,sampler=val_sampler,batch_size=batch_size)

In [ ]:
val_data = TensorDataset(val_seq,val_mask,val_y)

val_sampler = RandomSampler(val_data)

val_dataloader = DataLoader(val_data,sampler=val_sampler,batch_size=batch_size)

In [13]:
for param in bert.parameters():
  param.requires_grad=False

In [14]:
class Bert_Arch(nn.Module):
  def __init__(self,bert):
    super(Bert_Arch,self).__init__()

    self.bert= bert
    self.dropout = nn.Dropout(0.1)
    self.relu = nn.ReLU()


    self.fc1 = nn.Linear(768,512)
    self.fc2 = nn.Linear(512,2)
    self.softmax = nn.LogSoftmax(dim=1)


  def forward(self,sent_id,mask):

    _,cls_hs = self.bert(sent_id,attention_mask=mask)
    x = self.fc1(cls_hs)
    x = self.relu(x)
    x = self.dropout(x)

    x= self.fc2(x)
    x= self.softmax(x)

    return x












In [22]:
model = Bert_Arch(bert)
model = model.to(device)

In [16]:
from sklearn.utils.class_weight import compute_class_weight

class_wts = compute_class_weight('balanced',np.unique(train_label),train_label)
class_wts

array([0.65678244, 2.09456635])

In [17]:
weights = torch.tensor(class_wts,dtype = torch.float)
weights = weights.to(device)

cross_entropy = nn.NLLLoss(weight=weights)

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(),lr=1e-3)

In [ ]:
# function to train the model
def train():
  
  model.train()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save model predictions
  total_preds=[]
  
  # iterate over batches
  for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
    if step % 50 == 0 and not step == 0:
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

    # push the batch to gpu
    batch = [r.to(device) for r in batch]
 
    sent_id, mask, labels = batch

    # clear previously calculated gradients 
    model.zero_grad()        

    # get model predictions for the current batch
    preds = model(sent_id, mask)

    # compute the loss between actual and predicted values
    loss = cross_entropy(preds, labels)

    # add on to the total loss
    total_loss = total_loss + loss.item()

    # backward pass to calculate the gradients
    loss.backward()

    # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    # update parameters
    optimizer.step()

    # model predictions are stored on GPU. So, push it to CPU
    preds=preds.detach().cpu().numpy()

    # append the model predictions
    total_preds.append(preds)

  # compute the training loss of the epoch
  avg_loss = total_loss / len(train_dataloader)
  
  # predictions are in the form of (no. of batches, size of batch, no. of classes).
  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  #returns the loss and predictions
  return avg_loss, total_preds

In [ ]:
# function for evaluating the model
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = cross_entropy(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds

In [ ]:
epochs = 10

In [ ]:
# set initial loss to infinite
best_valid_loss = float('inf')

# empty lists to store training and validation loss of each epoch
train_losses=[]
valid_losses=[]

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    # append training and validation loss
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 10
  Batch    50  of    126.
  Batch   100  of    126.

Evaluating...

Training Loss: 0.530
Validation Loss: 0.515

 Epoch 2 / 10
  Batch    50  of    126.
  Batch   100  of    126.

Evaluating...

Training Loss: 0.330
Validation Loss: 0.219

 Epoch 3 / 10
  Batch    50  of    126.
  Batch   100  of    126.

Evaluating...

Training Loss: 0.312
Validation Loss: 0.468

 Epoch 4 / 10
  Batch    50  of    126.
  Batch   100  of    126.

Evaluating...

Training Loss: 0.292
Validation Loss: 0.283

 Epoch 5 / 10
  Batch    50  of    126.
  Batch   100  of    126.

Evaluating...

Training Loss: 0.278
Validation Loss: 0.264

 Epoch 6 / 10
  Batch    50  of    126.
  Batch   100  of    126.

Evaluating...

Training Loss: 0.229
Validation Loss: 0.167

 Epoch 7 / 10
  Batch    50  of    126.
  Batch   100  of    126.

Evaluating...

Training Loss: 0.262
Validation Loss: 0.160

 Epoch 8 / 10
  Batch    50  of    126.
  Batch   100  of    126.

Evaluating...

Training Loss: 0.192
Validat

In [23]:

#load weights of best model
path = 'saved_weights.pt'
model.load_state_dict(torch.load(path))




<All keys matched successfully>

In [29]:
# get predictions for test data
with torch.no_grad():
  preds = model(test_seq.to(device), test_mask.to(device))
  preds = preds.detach().cpu().numpy()

In [30]:
from sklearn.metrics import classification_report
preds = np.argmax(preds, axis = 1)
print(classification_report(test_y, preds))

              precision    recall  f1-score   support

           0       0.98      0.96      0.97       654
           1       0.89      0.92      0.91       206

    accuracy                           0.95       860
   macro avg       0.93      0.94      0.94       860
weighted avg       0.96      0.95      0.95       860



In [31]:
# confusion matrix
pd.crosstab(test_y, preds)

col_0,0,1
row_0,,
0,631,23
1,16,190


In [32]:
# We can clearly see that our model has done an outstanding job in predicitng whether the email is spam or ham.

In [33]:
# Since the files are very large there are chances that it your cuda wont work. Contact me at : prateekagrawaliiit@gmail.com and I can help you personally